# Calculating graph metrics with third party library

The purpouse of this notebook is to explore a third party library called [Graph-Stream](http://graphstream-project.org) for graph handling.

Test main features:
1. The test was implemented in [Scala 2.11](https://www.scala-lang.org/download/2.11.12.html)
2. The processing is handled by [Spark](https://github.com/apache/spark) a cluster computing framework.
3. The libraries required to run the test are the following:
    * [Cassandra connector](https://github.com/datastax/spark-cassandra-connector) (2.11 scala build version).
    * [Graph-stream](http://graphstream-project.org) (Java based library).
    * [Spark](https://mvnrepository.com/artifact/org.apache.spark/spark-core_2.11/2.2.1) (2.11 scala build version).
4. The Spark version used correspond to standalone application which mean that the use of multiple hosts ecosystem to test isn't needed.

## Step 1: Load libraries from Maven

In order to download the required libraries from the Maven repositories we need to use the following instructions (special Jupyter notebook's commands that allow Maven integration). In a traditional development environment we use a POM (Maven) or build.sbt (SBT) file to define the dependencies.

In [1]:
//Import dependencies from Maven
classpath.add("org.apache.spark" % "spark-core_2.11" % "2.1.1")
classpath.add("org.apache.spark" % "spark-sql_2.11" % "2.1.1")
classpath.add("com.datastax.spark" % "spark-cassandra-connector_2.11" % "2.0.0-M3")
classpath.add("org.graphstream" % "gs-core" % "1.3")
classpath.add("org.graphstream" % "gs-algo" % "1.3")


Adding 114 artifact(s)
Adding 13 artifact(s)
Adding 4 artifact(s)
Adding 5 artifact(s)
Adding 7 artifact(s)


## Step 2: Create a Spark context

Before start any computation we need to create a Spark context. 

The following code set up Spark's configuration:

In [2]:
import org.apache.spark.SparkConf
val configuration = new SparkConf()
    .setAppName("Graph-Stream test")
    .setMaster("local[*]") // Use all available computer's cpu cores
    .set("spark.executor.memory", "1g") // Amount of memory to use per executor process
    .set("spark.testing.memory", "2147480000")// Avoid any memory issues
    .set("spark.cassandra.connection.host", "127.0.0.1") // Defines the Cassandra host
    .set("spark.cassandra.auth.username", "cassandra") //Cassandra credentials
    .set("spark.cassandra.auth.password", "cassandra")

import org.apache.spark.SparkConf
configuration: org.apache.spark.SparkConf = org.apache.spark.SparkConf@8523ff

Once configured, the Spark context can be created.

The following code initialize the Spark context:

In [3]:
import org.apache.spark.{SparkContext}
val sc = new SparkContext(configuration)

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
18/11/13 08:31:23 INFO SparkContext: Running Spark version 2.1.1
18/11/13 08:31:31 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
18/11/13 08:31:35 WARN Utils: Your hostname, spark resolves to a loopback address: 127.0.1.1; using 192.168.1.207 instead (on interface eth0)
18/11/13 08:31:35 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
18/11/13 08:31:36 INFO SecurityManager: Changing view acls to: jose
18/11/13 08:31:36 INFO SecurityManager: Changing modify acls to: jose
18/11/13 08:31:36 INFO SecurityManager: Changing view acls groups to: 
18/11/13 08:31:36 INFO SecurityManager: Changing modify acls groups to: 
18/11/13 08:31:37 INFO SecurityManager: SecurityManager: authentication disabled; ui acls disabled; users  with view permissions: Set(jose); groups with view permissions: Set(); users  with modify per

import org.apache.spark.{SparkContext}
sc: org.apache.spark.SparkContext = org.apache.spark.SparkContext@1e15baa

## Step 3: Retrieve interactions from Cassandra

To populate the graph we retrieve the course interactions from Cassandra.

First we specify interaction's datatype:

In [4]:
//Interaccion datatype
case class interaccion(idinteraccion: String, atributos: Map[String, String]) 

defined class interaccion

The next lines of code read interactions from Cassandra and save into RDD (Resilient Distributed Dataset):

In [5]:
import com.datastax.spark.connector._ //Loads implicit functions
val interactions = sc.cassandraTable[interaccion]("diia", "interacciones") // Specify the Cassandra's keyspace and table
    .where("atributos['id_curso_origen']=?", 1) // Filter by id_curso = 1
    .collect() // Load the data into an RDD

import com.datastax.spark.connector._
interactions: Array[interaccion] = Array(
  interaccion(
    "1d004db3-fa60-41cd-9ca5-6fca4a7acb49",
    Map(
      "tipo_interaccion" -> "rea",
      "timestamp" -> "2007-10-24 08:20:07.263",
      "tipo_contenido" -> "des",
      "id_curso_origen" -> "1",
      "nodo_destino" -> "3",
      "contenido" -> "THANKFUL",
      "plataforma" -> "f",
      "id_curso_destino" -> "1",
      "sentimiento" -> "",
      "nodo_origen" -> "24"
    )
  ),
  interaccion(
    "cc6ea94b-d560-4fae-8549-c12559c91e38",
    Map(
      "tipo_interaccion" -> "men",
...

## Step 4: Build a graph from interactions

Graph-Stream offers mainly two ways of graph modeling:
* Single-graph: Is a graph that supports only one edge betweeen two nodes.
* Multi-graph: A graph that can have multiple edges between two nodes.

In this case we use a multi-graph because we need to differentiate in-degree and out-degree. Another aspect that we can take advantage is Graph-stream's features that permit graph auto creation only from the interactions set

The next code iterate over each interaction previously fetched to fill the Multi-graph:

In [6]:
import org.graphstream.graph.implementations.MultiGraph
import org.graphstream.graph.{Edge, Node}

val graph: MultiGraph = new MultiGraph("DIIA Graph")

// Enable graph auto-creation only from interactions
graph.setStrict(false)
graph.setAutoCreate(true)

interactions.foreach(
    interaction => graph.addEdge[Edge](interaction.idinteraccion.toString, interaction.atributos.get("nodo_origen").toString, interaction.atributos.get("nodo_destino").toString)
)
System.out.println("Number of Nodes->" + graph.getNodeCount + "\n")

Number of Nodes->46



import org.graphstream.graph.implementations.MultiGraph
import org.graphstream.graph.{Edge, Node}
graph: org.graphstream.graph.implementations.MultiGraph = DIIA Graph

Once modeled, we can get the degree of each node.

In [7]:
import scala.collection.JavaConverters._
graph.getNodeSet[Node].asScala.toArray.foreach( // Iterate over the graph's nodes
    // Show node's degree
    (n: Node) => System.out.println("nodeId->" + n.getId + "\n" + "inDegree->" + n.getInDegree + "\n" + "outDegree->" + n.getOutDegree + "\n")
)

nodeId->Some(24)
inDegree->11
outDegree->11

nodeId->Some(3)
inDegree->61
outDegree->61

nodeId->Some(19)
inDegree->81
outDegree->81

nodeId->Some(17)
inDegree->34
outDegree->34

nodeId->Some()
inDegree->346
outDegree->346

nodeId->Some(13)
inDegree->11
outDegree->11

nodeId->Some(29)
inDegree->3
outDegree->3

nodeId->Some(7)
inDegree->33
outDegree->33

nodeId->Some(14)
inDegree->38
outDegree->38

nodeId->Some(18)
inDegree->34
outDegree->34

nodeId->Some(4)
inDegree->39
outDegree->39

nodeId->Some(10)
inDegree->76
outDegree->76

nodeId->Some(5)
inDegree->40
outDegree->40

nodeId->Some(25)
inDegree->111
outDegree->111

nodeId->Some(16)
inDegree->49
outDegree->49

nodeId->Some(20)
inDegree->26
outDegree->26

nodeId->Some(21)
inDegree->28
outDegree->28

nodeId->Some(0)
inDegree->47
outDegree->47

nodeId->Some(22)
inDegree->44
outDegree->44

nodeId->Some(1)
inDegree->46
outDegree->46

nodeId->Some(9)
inDegree->16
outDegree->16

nodeId->Some(23)
inDegree->13
outDegree->13

nodeId->Some(11)


import scala.collection.JavaConverters._

## Step 5: Calculate the mainly centrality metrics

In this section we will calculate the main centrality metrics with the third party's build-in algorithms.

### Betweenness centrality
The betweenness centrality counts how many shortest paths between each pair of nodes of the graph pass by a node. It does it for all nodes of the graph. This measure might identify nodes with the ability to control information flow between different parts of the network.


In [8]:
import org.graphstream.algorithm.BetweennessCentrality

// Compute the betweenness centrality for each node in the graph.
val bcb = new BetweennessCentrality
bcb.init(graph)
bcb.compute()

System.out.println("Betweenness:")
// Iterate over each graph's node and show it calculated value
graph.getNodeSet[Node].asScala.toArray.foreach(
    (n: Node) => System.out.println(n.getId + "->" + n.getAttribute("Cb"))
)

Betweenness:
Some(24)->3.948859167811559
Some(3)->214.91816770214086
Some(19)->137.94334137917892
Some(17)->5.843250215552445
Some()->54.91488195864106
Some(13)->7.296179775925958
Some(29)->0.13008054522021847
Some(7)->25.486547416762484
Some(14)->41.28244434478991
Some(18)->45.999224049807836
Some(4)->46.0981327374728
Some(10)->87.98611020325387
Some(5)->16.494914794409716
Some(25)->9.13192175203748
Some(16)->70.73558697629242
Some(20)->1.6831236682655226
Some(21)->0.0
Some(0)->65.25229275949636
Some(22)->58.76973893881553
Some(1)->116.96977446394595
Some(9)->21.28107430317513
Some(23)->0.0
Some(11)->16.511937054767813
Some(34)->0.010221258539087452
Some(46)->0.02182085065551521
Some(26)->0.058103340942104176
Some(2)->74.83012409067176
Some(32)->0.0
Some(15)->87.1495471513281
Some(33)->0.10294117647058823
Some(27)->0.6815622105920432
Some(44)->0.0
Some(37)->0.07774576763340382
Some(30)->0.04793403066466102
Some(50)->0.022240364043704844
Some(6)->10.96686764237303
Some(35)->0.0
Some(28

import org.graphstream.algorithm.BetweennessCentrality
bcb: org.graphstream.algorithm.BetweennessCentrality = org.graphstream.algorithm.BetweennessCentrality@c5b73b

### Pagerank algorithm

The PageRank algorithm measures the "importance" of the nodes in a graph. It assigns to each node a rank. This rank corresponds to the probability that a "random surfer" visits the node.

In [9]:
import org.graphstream.algorithm.PageRank

// Initialization of the algorithm 
val pageRank = new PageRank
pageRank.init(graph)
pageRank.compute()

System.out.println("Pagerank:")
graph.getNodeSet[Node].asScala.toArray.foreach( // Iterave over each graph's node and print the calculated node's pagerank value
    (n: Node) => System.out.println(n.getId + "->" + n.getAttribute("PageRank"))
)

Pagerank:
Some(24)->0.010655649482737698
Some(3)->0.04694391120351182
Some(19)->0.05274289911538274
Some(17)->0.021791345457441536
Some()->0.20429061178450317
Some(13)->0.010407576841936775
Some(29)->0.005331746474090557
Some(7)->0.02464746772098131
Some(14)->0.02752087671217578
Some(18)->0.026085763592871097
Some(4)->0.029153790096185166
Some(10)->0.0481720583178849
Some(5)->0.025843894235493343
Some(25)->0.06645453914319224
Some(16)->0.032814932361962285
Some(20)->0.017211742323640006
Some(21)->0.01731296161404435
Some(0)->0.03311769577390839
Some(22)->0.03140799843118387
Some(1)->0.03416099735127152
Some(9)->0.014236612081643183
Some(23)->0.009785055159315627
Some(11)->0.01361290912561936
Some(34)->0.0045107663185433
Some(46)->0.006960687207748007
Some(26)->0.007050894004983111
Some(2)->0.035185060879378116
Some(32)->0.003836013511585187
Some(15)->0.028899268160889096
Some(33)->0.006544963372927964
Some(27)->0.007383734688100394
Some(44)->0.004434497368622707
Some(37)->0.00630456659

import org.graphstream.algorithm.PageRank
pageRank: org.graphstream.algorithm.PageRank = org.graphstream.algorithm.PageRank@1c31c60

### Eccentricity algorithm

The Eccentricity (node's largest geodesic) measure how far an node is from the furthest other.

This algorithm needs that APSP (All Pair Shortest Path) algorithm has been computed before its own computation. The following code calculate the shortest path from any node to any destination.

In [10]:
import org.graphstream.algorithm.APSP
val apsp = new APSP
apsp.init(graph)
apsp.setDirected(true)
apsp.compute()

import org.graphstream.algorithm.APSP
apsp: org.graphstream.algorithm.APSP = org.graphstream.algorithm.APSP@7c9cd4

Calculate the eccentricity algorithm:

In [11]:
import org.graphstream.algorithm.Eccentricity

val eccentricity = new Eccentricity
eccentricity.init(graph)
eccentricity.compute()
System.out.println("Eccentricity:")
graph.getNodeSet[Node].asScala.toArray.foreach(
    (n: Node) => System.out.println(n.getId + "->" + n.getAttribute("eccentricity"))
)

Eccentricity:
Some(24)->false
Some(3)->false
Some(19)->true
Some(17)->false
Some()->true
Some(13)->false
Some(29)->false
Some(7)->false
Some(14)->true
Some(18)->false
Some(4)->false
Some(10)->true
Some(5)->false
Some(25)->true
Some(16)->false
Some(20)->false
Some(21)->false
Some(0)->true
Some(22)->false
Some(1)->true
Some(9)->false
Some(23)->false
Some(11)->false
Some(34)->false
Some(46)->false
Some(26)->false
Some(2)->false
Some(32)->false
Some(15)->true
Some(33)->false
Some(27)->false
Some(44)->false
Some(37)->false
Some(30)->false
Some(50)->false
Some(6)->false
Some(35)->false
Some(28)->false
Some(42)->false
Some(38)->false
Some(45)->false
Some(36)->false
Some(43)->false
Some(40)->false
Some(31)->false
Some(12)->false


import org.graphstream.algorithm.Eccentricity
eccentricity: org.graphstream.algorithm.Eccentricity = org.graphstream.algorithm.Eccentricity@a1e145

Closeness centrality measures the proximity of an node to the other nodes in the social network.

In [12]:
// Calculate and show node's closeness
import org.graphstream.algorithm.measure.ClosenessCentrality
val closeness: ClosenessCentrality = new ClosenessCentrality ()
closeness.init(graph)
closeness.compute()
System.out.println("Closenness:")
graph.getNodeSet[Node].asScala.toArray.foreach(
    (n: Node) => System.out.println(n.getId + "->" + n.getAttribute("closeness"))
)

Closenness:
Some(24)->0.011764705882352941
Some(3)->0.015151515151515152
Some(19)->0.014925373134328358
Some(17)->0.011494252873563218
Some()->0.015151515151515152
Some(13)->0.009708737864077669
Some(29)->0.009708737864077669
Some(7)->0.012048192771084338
Some(14)->0.0136986301369863
Some(18)->0.01282051282051282
Some(4)->0.012987012987012988
Some(10)->0.014084507042253521
Some(5)->0.011904761904761904
Some(25)->0.014492753623188406
Some(16)->0.013333333333333334
Some(20)->0.010869565217391304
Some(21)->0.00909090909090909
Some(0)->0.0136986301369863
Some(22)->0.013888888888888888
Some(1)->0.014925373134328358
Some(9)->0.011627906976744186
Some(23)->0.00909090909090909
Some(11)->0.011235955056179775
Some(34)->0.008695652173913044
Some(46)->0.011111111111111112
Some(26)->0.01098901098901099
Some(2)->0.013157894736842105
Some(32)->0.008333333333333333
Some(15)->0.014705882352941176
Some(33)->0.010416666666666666
Some(27)->0.010526315789473684
Some(44)->0.00909090909090909
Some(37)->0.010

import org.graphstream.algorithm.measure.ClosenessCentrality
closeness: org.graphstream.algorithm.measure.ClosenessCentrality = org.graphstream.algorithm.measure.ClosenessCentrality@11f11fe

# Algorithm implementation

In order to take advantage of Graph-Stream's features like APSP, we will implement our own version of closeness metric. 

In [13]:
System.out.println("\nCustom closenness implementation:")
import org.graphstream.algorithm.APSP.APSPInfo

graph.getNodeSet[Node].asScala.toArray.foreach(
    (x: Node) => {
      var sum = 0.0

      graph.getNodeSet[Node].asScala.toArray.foreach(
        (y: Node) => {
          if (!x.getId.equals(y.getId)) {
            val info:APSPInfo = x.getAttribute[APSPInfo](APSP.APSPInfo.ATTRIBUTE_NAME)
            sum += info.getShortestPathTo(y.getId).getEdgeCount
          }
        }
      )

      val closenness = (1 / sum)
      System.out.println(x.getId+ "->" + closenness)
    }
)


Custom closenness implementation:
Some(24)->0.011764705882352941
Some(3)->0.015151515151515152
Some(19)->0.014925373134328358
Some(17)->0.011494252873563218
Some()->0.015151515151515152
Some(13)->0.009708737864077669
Some(29)->0.009708737864077669
Some(7)->0.012048192771084338
Some(14)->0.0136986301369863
Some(18)->0.01282051282051282
Some(4)->0.012987012987012988
Some(10)->0.014084507042253521
Some(5)->0.011904761904761904
Some(25)->0.014492753623188406
Some(16)->0.013333333333333334
Some(20)->0.010869565217391304
Some(21)->0.00909090909090909
Some(0)->0.0136986301369863
Some(22)->0.013888888888888888
Some(1)->0.014925373134328358
Some(9)->0.011627906976744186
Some(23)->0.00909090909090909
Some(11)->0.011235955056179775
Some(34)->0.008695652173913044
Some(46)->0.011111111111111112
Some(26)->0.01098901098901099
Some(2)->0.013157894736842105
Some(32)->0.008333333333333333
Some(15)->0.014705882352941176
Some(33)->0.010416666666666666
Some(27)->0.010526315789473684
Some(44)->0.0090909090

import org.graphstream.algorithm.APSP.APSPInfo

In [14]:
// free spark's resources
sc.stop()

## Conclusions:

* Graph-stream can interoperate with the Scala language and the Spark framework without problems.
* The third party offers a wide variety of build-in algorithms.
* It brings the possibility to implement additional algorithms that take advantage of features like APSP.
